# The Ising Model
## Ferromagnetism and Paramagnetism
When we think of magnets, the classic refrigerator magnet is probably the first thing to come to mind.

In [1]:
{
    "tags": [
        "hide-input",
    ]
}

from IPython.display import IFrame

IFrame(src='https://www.youtube.com/embed/u36QpPvEh2c', width='800', height='450')